## ひろしまQuest2020#stayhome【球種予測部門】
#### all_merge_17
- イニング時間、打席時間、平均投球間隔(差)、サヨナラ、延長戦

#### all_merge_16
- 球団ごとの年棒ランク
- 選手IDごとの2017年のコース実績

#### all_merge_15
- 左右で球種/コースを別に集計

#### all_merge_14
- 左右で球種を別に集計

#### all_merge_13
- 前の投球からの時間差

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_17.f'
SUBMIT = 'submit/ball_17.csv'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 192)

In [4]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 192)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_course00,bc_course01,bc_course02,bc_course03,bc_course04,bc_course05,bc_course06,bc_course07,bc_course08,bc_course09,bc_course10,bc_course11,bc_course12,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,...,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0,0,0,0,0,1,0,0,0.000000,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0,0,0,0,0,1,0,0,0.200000,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
2,2,0.0,5.0,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0,0,0,0,0,1,0,0,0.383333,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
3,3,0.0,12.0,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0,0,0,0,0,1,0,0,1.016667,...,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1
4,4,0.0,8.0,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.463496,0.099763,0.194714,0.067542,0.091198,0.023901,0.004486,0.054898,0.024390,0.034832,0.025124,0.039481,0.047557,0.033526,0.049596,0.080512,0.069337,0.085243,0.121054,0.076923,0.312424,0,0,0,0,0,1,0,0,1.200000,...,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1


### train
- 行数: 257117

In [5]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 192)

### test
- 行数: 521650

In [6]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 192)

In [7]:
del all_pitch
gc.collect()

60

In [8]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [9]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

## Dataset作成

In [10]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [11]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 6.9923570049658075, 
        'lambda_l2': 0.002378623984798833, 
        'num_leaves': 18, 
        'feature_fraction': 0.45199999999999996, 
        'bagging_fraction': 0.9799724836460725, 
        'bagging_freq': 4, 
        'min_child_samples': 20
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 1.14225 + 0.00203689
[200]	cv_agg's multi_logloss: 1.12108 + 0.00227054
[300]	cv_agg's multi_logloss: 1.11488 + 0.00212037
[400]	cv_agg's multi_logloss: 1.11131 + 0.00215743
[500]	cv_agg's multi_logloss: 1.10855 + 0.00231498
[600]	cv_agg's multi_logloss: 1.10676 + 0.00247626
[700]	cv_agg's multi_logloss: 1.10532 + 0.00255608
[800]	cv_agg's multi_logloss: 1.10447 + 0.00263771
[900]	cv_agg's multi_logloss: 1.10379 + 0.0027525
[1000]	cv_agg's multi_logloss: 1.1035 + 0.00276765
[1100]	cv_agg's multi_logloss: 1.10341 + 0.00293211
[1200]	cv_agg's multi_logloss: 1.10352 + 0.00312714
Best num_boost_round: 1100
Best CV score: 1.1034107856339535
CPU times: user 2h 10min 54s, sys: 1min 53s, total: 2h 12min 48s
Wall time: 26min 44s


In [12]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [13]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 31min, sys: 5.87 s, total: 31min 6s
Wall time: 3min 53s


### Feature Importance

In [14]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
103,shoot,1451
164,salary_year_dif_p-b,1452
127,weight_bat,1477
176,bmi_dif_p-c,1479
64,point_diff,1589
101,curve,1591
167,rank_x_year_dif_p-b,1609
104,fork,1644
182,rank_year_dif_b-c,1699
180,salary_year_dif_b-c,1717


In [15]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 10s, sys: 601 ms, total: 6min 10s
Wall time: 47.5 s


## 結果出力

In [16]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.458827,0.038079,0.153396,0.249397,0.003439,0.000022,0.000023,0.096817
1,1,0.535311,0.089188,0.111187,0.213810,0.005219,0.000015,0.000005,0.045264
2,2,0.332822,0.110164,0.160430,0.133441,0.140586,0.000104,0.000028,0.122426
3,3,0.379061,0.135033,0.186936,0.215217,0.020283,0.000048,0.000013,0.063411
4,4,0.193682,0.055310,0.348981,0.260257,0.020060,0.000050,0.000015,0.121645


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [17]:
submit.mean()

index    260824.500000
0             0.513238
1             0.074392
2             0.187077
3             0.052872
4             0.071364
5             0.047480
6             0.006381
7             0.047196
dtype: float64

In [18]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [19]:
'signate submit --competition-id=275 ./{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=275 ./submit/ball_17.csv --note feat=192_cv=1.1034107856339535'

### GCS

In [20]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/ball_17.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 17 (191) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1034107856339535
- Wall time: 26min 44s
- 評価結果  : 1.3963057

#### 16 (181) ... 選手IDごとの2017年のコース実績
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1178
- Best CV score: 1.1021930539837317
- Wall time: 12min 35s
- 評価結果  : 1.3993836

#### 15 (153) ... 左右で球種/コースを別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1189
- Best CV score: 1.1013869435441306
- Wall time: 11min 9s
- 評価結果  : 未提出

#### 14 (153) ... 左右で球種を別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1240
- Best CV score: 1.1021952294615331
- Wall time: 11min 34s
- 評価結果  : 1.3981900

#### 13 (157) ... 前の投球からの時間差を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1416
- Best CV score: 1.1062463941695753
- Wall time: 13min 10s
- 評価結果  : 1.4012184

#### 12 (156) ... コースの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1353
- Best CV score: 1.106942086205103
- Wall time: 27min 41s
- 評価結果  : 1.4016077

#### 11 (143) ... ボールカウントの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1209
- Best CV score: 1.1075153721728936
- Wall time: 23min 19s
- 評価結果  : 未提出

#### 10_tune (135) ... 再チューニング
- 'num_leaves' : 15,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1260
- Best CV score: 1.1118897437428692
- Wall time: 8min 40s
- 評価結果  : 1.4060397

#### 10s (135) ... trainを75%サンプリング
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1412
- Best CV score: 1.1190287120710383
- Wall time: 12min 5s
- 評価結果  : 1.4310869

#### 10 (135) ... 5相当に戻した
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1112
- Best CV score: 1.111265000190272
- Wall time: 9min 30s
- 評価結果  : 1.4008712

#### 9 (136) ... 5相当に戻した=>NG
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1106
- Best CV score: 1.1113177029272225
- Wall time: 25min 44s (local)
- 評価結果  : 1.5133521

#### 8 (157)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1093728644236824
- Wall time: 10min 4s
- 評価結果  : 1.4925832

#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806